In [3]:
from topsolid_api import TopSolidAPI

api = TopSolidAPI()

if api.connected:
    print("Connected to TopSolid")
    
    lang = api.get_language()

    print(api.ts.get_Version() , lang)

    licc = api.ts.Licenses.GetActveLicenses()
    print(len(licc))

    for l in licc:
        print(l.Name, l.Status, l.ExpirationDate, l.LicensedTo, l.LicenseType, l.Module, l.Provider, l.Version)

    # Get the current document
    doc = api.get_current_project()
    print(doc)


Connected to TopSolid
718400000 fr-FR
1
TopSolid'All Valid 09/11/2050 22:59:59 TOPSOLID Floating 0 TOPSOLID 7.18
None


In [4]:
from topsolid_api import TopSolidAPI

api = TopSolidAPI()

#get the current document
doc = api.get_current_project()
if doc:
    print(f"{doc[1]} - {doc[0].Id}")
else:
    print("No document open")

No document open


In [5]:
import os
from topsolid_api import TopSolidAPI

api = TopSolidAPI()

# Search in directory for all toppkg files and import them
path = "C:\\Users\\user\\Downloads"

# List all files in the directory with the .toppkg extension
files = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.TopPkg')]

print(len(files))

for f in files:
    print(f)
    api.ts.Pdm.ImportPackageAsReplication (f)




3
C:\Users\user\Downloads\41383.TopPkg
C:\Users\user\Downloads\P046787.TopPkg
C:\Users\user\Downloads\P600289.TopPkg


In [6]:
import os
import wx
from topsolid_api import TopSolidAPI

api = TopSolidAPI()

app = wx.App(False)


dialog = wx.DirDialog(None, "Select the folder containing the .TopPkg files", "", wx.DD_DEFAULT_STYLE | wx.DD_DIR_MUST_EXIST)

path = ""
if dialog.ShowModal() == wx.ID_OK:
    path = dialog.GetPath()
dialog.Destroy()

# Verifica se um caminho foi selecionado
if path:
    # Lista todos os arquivos no diretório com a extensão .TopPkg
    files = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.TopPkg')]

    for f in files:
        print(f)
        api.ts.Pdm.ImportPackageAsReplication(f)
else:
    print("Nenhuma pasta foi selecionada.")

Nenhuma pasta foi selecionada.


In [ ]:
import sys
!{sys.executable} -m pip install trimesh
!{sys.executable} -m pip install gmsh-sdk




In [1]:
import trimesh
import warnings
warnings.filterwarnings("ignore")

In [1]:
import trimesh.interfaces


mesh = trimesh.Trimesh(**trimesh.interfaces.gmsh.load_gmsh(file_name = "cube.stp", gmsh_args = [
    ("Mesh.Algorithm", 12),
    ("Mesh.CharacteristicLengthFromCurvature", 20),
    ("General.NumThreads", 10),
    ("Mesh.MinimumCirclePoints", 20)]))

#print(mesh.volume)
#scene = mesh.scene()
#scene.show()
print("Done")

# Save the mesh to a file stl
mesh.export("cube.stl")

Done


b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xe0?\x00\x00\x00\x00\x80?\x8aRJ\xa9L\x17\xe9\xa8\x1f\x05\xcaB\xdcO\x07\xc0\xdcO\x07\xc0\x1f\x05\xcaB;\xa2\xd1\xbf\x12\xec\xeb\xbf\x1f\x05\xcaB$O\x03\xc0=;\xd6\xbf\x00\x00\x00\x00\x80?\x91ZV)\x95\xb2/\xa9\x1f\x05\xcaB\xdcO\x07\xc0\xdcO\x07\xc0\x1f\x05\xcaB$O\x03\xc0=;\xd6\xbf\x1f\x05\xcaBA\xd0\x1a\xc0\t\xf5\xe0\xbf\x00\x00\x00\x00\x80?\xff\xcf`(\xe4I\t)\x1f\x05\xcaB\xdcO\x07@\xdcO\x07@\x1f\x05\xcaB;\xa2\xd1?\x12\xec\xeb?\x1f\x05\xcaB$O\x03@=;\xd6?\x00\x00\x00\x00\x80?\x91ZV\xa9\x95\xb2/)\x1f\x05\xcaB\xdcO\x07@\xdcO\x07@\x1f\x05\xcaB$O\x03@=;\xd6?\x1f\x05\xcaBA\xd0\x1a@\t\xf5\xe0?\x00\x00\x00\x00\x80?\xcb\xb3\xa6\xa9\x99\x81\xe0\xa9\x1f\x05\xcaB\'M\xec\xbfn\x84\x1e?\x1f\

In [1]:
import gmsh
import math

# Initialize Gmsh
gmsh.initialize()
gmsh.model.add("SlicedModel")

# Load the shape from a file
gmsh.model.occ.importShapes("cube.stp")
gmsh.model.occ.synchronize()

# Get the bounding box of the volume
entities = gmsh.model.getEntities(3)
bbox = gmsh.model.getBoundingBox(entities[0][0], entities[0][1])
xmin, ymin, zmin, xmax, ymax, zmax = bbox

# Define constants and parameters
N = 5  # Number of slices
dir = 0  # Direction: 0 = X, 1 = Y, 2 = Z
surf = 0  # Keep only surfaces?

dx = xmax - xmin
dy = ymax - ymin
dz = zmax - zmin
L = dz if dir == 0 else dx
H = dz if dir == 1 else dy

# Create the first cutting plane
s = []
s.append(gmsh.model.occ.addRectangle(xmin, ymin, zmin, L, H))
if dir == 0:
    gmsh.model.occ.rotate([(2, s[0])], xmin, ymin, zmin, 0, 1, 0, -math.pi / 2)
elif dir == 1:
    gmsh.model.occ.rotate([(2, s[0])], xmin, ymin, zmin, 1, 0, 0, math.pi / 2)

tx = dx / N if dir == 0 else 0
ty = dy / N if dir == 1 else 0
tz = dz / N if dir == 2 else 0
gmsh.model.occ.translate([(2, s[0])], tx, ty, tz)

# Create the other cutting planes
for i in range(1, N - 1):
    new_s = gmsh.model.occ.copy([(2, s[0])])[0][1]
    gmsh.model.occ.translate([(2, new_s)], i * tx, i * ty, i * tz)
    s.append(new_s)

# Fragment (i.e. intersect) the volume with all the cutting planes
volumes = gmsh.model.occ.fragment([(3, entities[0][1])], [(2, surf) for surf in s])
gmsh.model.occ.synchronize()

# Remove all the surfaces that are not on the boundary of a volume
gmsh.model.occ.remove([(2, surf) for surf in s])
gmsh.model.occ.synchronize()

if surf:
    # If we want to only keep the surfaces, retrieve the surfaces in bounding boxes around the cutting planes
    eps = 1e-4
    s = []
    for i in range(1, N):
        xx = xmin if dir == 0 else xmax
        yy = ymin if dir == 1 else ymax
        zz = zmin if dir == 2 else zmax
        s += gmsh.model.getEntitiesInBoundingBox(
            xmin - eps + i * tx, ymin - eps + i * ty, zmin - eps + i * tz,
            xx + eps + i * tx, yy + eps + i * ty, zz + eps + i * tz, 2
        )
    # Remove all the other entities
    all_surfaces = gmsh.model.getEntities(2)
    dels = [surf for surf in all_surfaces if surf not in s]
    gmsh.model.occ.remove(dels)
    gmsh.model.occ.remove(gmsh.model.getEntities(3))
    gmsh.model.occ.remove(gmsh.model.getEntities(1))
    gmsh.model.occ.remove(gmsh.model.getEntities(0))
    gmsh.model.occ.synchronize()

# Specify a global mesh size
gmsh.option.setNumber("Mesh.MeshSizeMin", 3)
gmsh.option.setNumber("Mesh.MeshSizeMax", 3)

# Generate the mesh
gmsh.model.mesh.generate(3)

# Save the mesh to a file
gmsh.write("sliced_model.msh")

# Finalize Gmsh
gmsh.finalize()

# Open the mesh in Gmsh GUI for visualization
import os
os.system("gmsh sliced_model.msh")

1

In [3]:


modelDim = mesh.extents
#get the dimensions of the model
mDimX = modelDim.item(0)
mDimY = modelDim.item(1)
mDimZ = modelDim.item(2)

print(mDimX, mDimY, mDimZ, modelDim)

NameError: name 'mesh' is not defined

In [2]:
#get open projects
projs = api.ts.Pdm.GetOpenProjects(True, True)
for d in projs:
    name = api.ts.Pdm.GetName(d)
    print(f"{name} - {d.Id}")

NameError: name 'api' is not defined

In [7]:
#get open documents
docs = api.ts.Documents.GetOpenDocuments()
for d in docs:
    name = api.get_name(d) # api.ts.Documents.GetName(d)
    print(f"{name} - {d.PdmDocumentId}")


x35396.64543.47757.45227_2024-07-29-003848_JS755060E2C.0Z5-HXT_Basic - 19_422a7329-adf6-4b2d-8d83-ca4a63594232&2_1238_0_0
V7470450 - 19_422a7329-adf6-4b2d-8d83-ca4a63594232&2_1232_0_1
x35396.64543.47757.45227_2024-07-29-003848_JS755060E2C.0Z5-HXT_Basic - 19_422a7329-adf6-4b2d-8d83-ca4a63594232&2_1239_0_0


In [8]:
test = api.get_current_project()
if test:
    print(f"{test[1]} - {test[0].Id}")

In [9]:
#void CloseAll(bool inAsksUser,bool inSaves)
api.ts.Documents.CloseAll(False, False)

#void GetConstituents(PdmObjectId inObjectId,out List<PdmObjectId> outFolderIds,out List<PdmObjectId> outDocumentObjectIds)
for p in projs:
    sd = api.ts.Pdm.GetConstituents(p)
    print(sd)
    for d in sd:
        print(f"{d[0].Id} - {api.get_name(d[0])}")
        for f in d:
            print(f"{f.Id} - {api.get_name(f)} - {api.get_type(f)}")
            

NameError: name 'projs' is not defined

In [10]:
all = api.get_constituents(p, True)
print(all)

NameError: name 'p' is not defined

In [11]:
proj = api.get_open_files()
for p in proj:
    
    qs = api.get_Frames(p)

number of open files : 0


In [12]:
for i in range(0, 40):
    uop = ""
    out = ""
    opt = api.ts.Application.GetImporterFileType(i, uop, out)
    print(i, opt, out)
    opts = api.get_importer_options(i, True)
    for o in opts:
        print(o.Key, o.Value)


0 ('3D XML (Spatial)', <System.String[] object at 0x000002AB40C42EC0>) 
opt :: System.Collections.Generic.List`1[TopSolid.Kernel.Automating.KeyValue] :: 0
1 ('Acis (Spatial)', <System.String[] object at 0x000002AB5A118500>) 
opt :: System.Collections.Generic.List`1[TopSolid.Kernel.Automating.KeyValue] :: 14
TRANSLATES_ASSEMBLY False
TRANSLATES_ATTRIBUTES True
SIMPLIFIES_GEOMETRY True
SEWS_SHEETS True
LINEAR_TOLERANCE 1E-05
BODY_DOCUMENT_EXTENSION .TopPrt
ASSEMBLY_DOCUMENT_EXTENSION .TopAsm
TRANSLATES_FREE_CURVES True
TRANSLATES_FREE_SURFACES True
TRANSLATES_HIDDEN_ENTITIES False
IMPORTS_PMI False
TRANSLATES_MATERIAL False
DEEP_HEALING False
IS_TRANSLATOR_SILENT False
2 ('Catia V4 (Spatial)', <System.String[] object at 0x000002AB5A1188C0>) 
opt :: System.Collections.Generic.List`1[TopSolid.Kernel.Automating.KeyValue] :: 14
TRANSLATES_ASSEMBLY False
TRANSLATES_ATTRIBUTES True
SIMPLIFIES_GEOMETRY True
SEWS_SHEETS True
LINEAR_TOLERANCE 1E-05
BODY_DOCUMENT_EXTENSION .TopPrt
ASSEMBLY_DOCUMEN

In [13]:
a = 123
b = 321
print(a, b)
a = a ^ b
print(a, b)
b = a ^ b
print(a, b)
a = a ^ b
print(a, b)

123 321
314 321
314 123
321 123
